In [23]:
import cv2
import numpy as np
import argparse
import csv
import os
import math
import random
import time
from copy import deepcopy
from operator import itemgetter
from itertools import *


In [24]:
def make_ellipse(image, row):
    center = (int(round(row[0])),int(round(row[1])))
    axes = (int(round(row[2])*2 ),int(round(row[3])*2) )
    angle = int(round(row[4]))
    color = (row[8],row[9],row[10])
    thickness = 4  #Do -1 if you want it filled in

    return cv2.ellipse(image, center, axes, angle, 0, 360, color, thickness) 

def make_ellipse2(image, ellipse):
    thickness = 4  #Do -1 if you want it filled in
    
    return cv2.ellipse(image, ellipse[1], ellipse[3], thickness) 

def make_ellipse3(image, ellipse,tubenum):
    thickness = 4  #Do -1 if you want it filled in
    center = (int(ellipse[0][1][0][0]), int(ellipse[0][1][0][1]) )
    
    cv2.putText(image,str(tubenum), center,cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), thickness)
    return cv2.ellipse(image, ellipse[0][1], ellipse[0][3], thickness) 

def draw_event(image, event):
    thickness = 4
    center = (int(event[2]), int(event[4]) )
    
    return cv2.putText(image,"X", center,cv2.FONT_HERSHEY_PLAIN, 4, (0,0,255), thickness)
    

In [25]:
#This code is a recreation of the original tube code in Python. The input is a magno video and the output is a list of tubes

inmagnovid = 'int6_magno.mp4'

vidcap = cv2.VideoCapture(inmagnovid) 
kernel = np.ones((10,10) , np.uint8)
magnovid = []
H=0
W=0
F=0

#Thresholds (at 50) the input magno video into a binary video 'magnovid'
while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        (dontcare, outframe) = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY)
        cv2.dilate(outframe,kernel, iterations =1)
        cv2.erode(outframe,kernel, iterations =1)
        H = frame.shape[0]
        W = frame.shape[1]
        F=F+1
        magnovid.append(outframe)

vidcap.release()
cv2.destroyAllWindows()


#Write the input video into a variable
invideo = 'int6.mp4'
vidcap = cv2.VideoCapture(invideo) 
invid = []

while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        invid.append(frame)
        
vidcap.release()
cv2.destroyAllWindows()


#Writes the thresholded magno vid to a video file
vidwriter = cv2.VideoWriter('int6_magnoOut.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%50==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnovid[i])
vidwriter.release()
cv2.destroyAllWindows()


#Each ellipse is put in the list as a tuple... (framenum, ellipse)
#Ellipse: (x,y) , (MAJORAXIS,minoraxis), angle
eList = []
for f in range(1,F): #for each frame
    M = cv2.cvtColor(magnovid[f], cv2.COLOR_BGR2GRAY);
    _, contours, hierarchy = cv2.findContours(M, 1, 2)
    
    for c in contours:
        if len(c) < 5:
            continue
        area = cv2.contourArea(c)
        if area < 100:
            continue
    
        ellipse = cv2.fitEllipse(c)
        eList.append( [f, ellipse, [0,0], [255,255,255] ] )
        

#Filtering the smallest ellipses (minsize for either dimension = 20pixels)
minsize = 20
eList = [e for e in eList if (e[1][1][0] > minsize) and (e[1][1][1]>minsize)]



#Draw the ellipses on the video just to check...
magnotubes = deepcopy(invid)
for f in range(1,F):
    puthere = [e for e in eList if e[0]==f]
    
    for ellipse in puthere:
        make_ellipse2(magnotubes[f],ellipse)
        

#Writes the vid with tubes to a video file
vidwriter = cv2.VideoWriter('int6_outstage0.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%100==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnotubes[i])
vidwriter.release()
cv2.destroyAllWindows()

FRAME 50 / 540
FRAME 100 / 540
FRAME 150 / 540
FRAME 200 / 540
FRAME 250 / 540
FRAME 300 / 540
FRAME 350 / 540
FRAME 400 / 540
FRAME 450 / 540
FRAME 500 / 540
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [26]:


#Doing the first temporal associations. Should start to look like tubes
initialList = []
finalList = []
paths = [[]]
istart = 6
iend = istart+4
fstart = 11
fend = fstart+5
locthresh = 80
sizethresh = 50

while (fend < F):
    initialList = [e for e in eList if e[0]>=istart and e[0]<=iend]
    finalList =  [e for e in eList if e[0]>=fstart and e[0]<=fend]
    
    for ei in initialList:
        for ef in finalList:
            
            matchFlag = False
            r = math.sqrt((ef[1][0][0] - ei[1][0][0])**2 + (ef[1][0][1] - ei[1][0][1])**2 )
            sizediff = ef[1][1][1] - ei[1][1][1] #diff in major axis
            framediff = ef[0]-ei[0]
            
            if not(ei[2]==(0,0)):
                predX = ei[2][0]*framediff + ei[1][0][0]
                predY = ei[2][1]*framediff + ei[1][0][1]
                if (abs(predX-ef[1][0][0]) < 50) and (abs(predY-ef[1][0][1]) < 50):
                    matchFlag = True
            else:
                if (abs(sizediff)<sizethresh) and (r<locthresh):
                    matchFlag = True
            
            if (matchFlag):
                if ei[3]==[255,255,255]:
                    ei[3][0] = random.randint(0,255)
                    ei[3][1] = random.randint(0,255)
                    ei[3][2] = random.randint(0,255)
                ef[3][0] = ei[3][0]
                ef[3][1] = ei[3][1]
                ef[3][2] = ei[3][2]
                
                ef[2][0] = (ef[1][0][0]-ei[1][0][0])/framediff
                ef[2][1] = (ef[1][0][1]-ei[1][0][1])/framediff
                

                #This part searches for ei in existing paths and attaches ef if it finds it
                whichpath = 0
                for i in range(1,len(paths)): 
                    for ell in paths[i]:
                        if ei == ell:
                            whichpath = i
                            
                if not(whichpath==0):
                     paths[whichpath].append(ef)
                else:
                    newpath = []
                    newpath.append(ei)
                    newpath.append(ef)
                    paths.append(newpath)

    #print(fend)
    istart=istart+5
    iend=iend+5
    fstart=fstart+5
    fend=fend+5

def putTubesOnVid(tubevar, vidoutfile, withnums = False):
    
    if not withnums:
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                puthere.extend(P)

            for ellipse in puthere:
                make_ellipse2(outvid[f],ellipse)

        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    
    else:        
        #WITH NUMS    
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                T = [tubevar.index(path) for e in path if e[0]==f]
                puthere.append([P,T])

            for [ellipse,Tnum] in puthere:
                if ellipse:
                    make_ellipse3(outvid[f],ellipse,Tnum)


        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    
    
def putEventsOnVid(eventvar, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Draw the events on the video to get a visual
    outvid = deepcopy(inputvid)
    for f in range(1,F):
        puthere = [e for e in eventvar if (f>=e[0]) and (f<=e[1])]
        for event in puthere:
            draw_event(outvid[f],event)

    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()
    
    
putTubesOnVid(paths, 'int6_outstage1.mp4')

FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [27]:
#Fixes the paths so that each color corresponds to a path
newpaths=[[]]

for i in range(1,len(paths)):
    for e in paths[i]:
        RGB = e[3]
        makePathFlag = True
        
        for newpath in newpaths:
            if not newpath or not(newpath[0][3] == RGB):
                continue
            else:
                newpath.append(e)
                makePathFlag = False
                break
        
        if (makePathFlag):
            E = [e]
            newpaths.append(E)

            
#Gets rid of duplicates in each path (took forever smh)
K = deepcopy(newpaths)
L = deepcopy(newpaths)
M = deepcopy(newpaths)
newpaths = []

for i in range(0,len(K)):
    bigA = []
    for j in range(0,len(K[i])):
        thise = K[i][j]
        A = [K[i].index(e) for e in K[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [K[i][t] for t in list(set(bigA))]
    newpaths.append(out)

#for i in range(0,len(K)):
#    print("used to be...", len(K[i]))
#    print("now is...", len(newpaths[i]))


#Handles when there are 2 ellipses from the same tube on a single frame.
#We choose the bigger one
L = deepcopy(newpaths)
newpaths = []

for i in range(0,len(L)):
    toRemove = []
    for f in range(1,F):
        #build a list of shit to delete
        A = [e for e in L[i] if e[0] == f]
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) 
            toRemove.extend(A[:-1])

    newpaths.append([p for p in L[i] if p not in toRemove])
    

putTubesOnVid(newpaths, 'int6_outstage2.mp4')


FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [28]:
L = deepcopy(newpaths)
temporalcutoff = 20
newpaths = [p for p in L if len(p)> temporalcutoff]

for path in newpaths:
    path.sort(key = lambda x: x[0])

putTubesOnVid(newpaths, 'int6_outstage3.mp4')

FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [29]:
L = deepcopy(newpaths)
tubes = []

#Each slice should be [f, ((X,Y),(b,A),rot), [Vxavg,Vyavg], [R,G,B]]

for i in range(0,len(L)):
    slicestoAdd = []
    for f in range(L[i][0][0], L[i][-1][0]):
        #chunk = [L[i].index(e) for e in L[i] if abs(e[0]-f) < 15]
        
        thisone = [e for e in newpaths[1] if e[0]==f]
        if not thisone: #if we don't have a slice at this frame
            thischunk = []
            a = 2
            while len(thischunk) < 2:
                thischunk = [L[i].index(e) for e in L[i] if abs(e[0]-f)<=a]
                a=a+2
                
            chunkX = [L[i][ind][1][0][0] for ind in thischunk]
            X = sum(chunkX)/len(chunkX)
            chunkY = [L[i][ind][1][0][1] for ind in thischunk]
            Y = sum(chunkY)/len(chunkY)
            chunksmallax = [L[i][ind][1][1][0] for ind in thischunk]
            smallax=max(chunksmallax)
            chunkbigax = [L[i][ind][1][1][1] for ind in thischunk]
            bigax=max(chunkbigax)
            chunkphi = [L[i][ind][1][2] for ind in thischunk]
            phi=max(chunkphi)
            chunkf = [L[i][ind][0] for ind in thischunk]
            Vx = (chunkX[-1] - chunkX[0])/(max(chunkf)-min(chunkf))
            Vy = (chunkY[-1] - chunkY[0])/(max(chunkf)-min(chunkf))
        
            thisslice = [f, ((X,Y),(smallax,bigax),phi), [Vx,Vy], L[i][0][3]]
            slicestoAdd.append(thisslice)
            
    L[i].extend(slicestoAdd)
            
tubes = L


#Handles when there are 2 ellipses from the same tube on a single frame.
#We choose the bigger one
L = deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    toRemove = []
    for f in range(1,F):
        #build a list of shit to delete
        A = [e for e in L[i] if e[0] == f]
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) 
            toRemove.extend(A[:-1])

    tubes.append([p for p in L[i] if p not in toRemove])
    
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])
    
putTubesOnVid(tubes, 'int6_outstage4.mp4')
putTubesOnVid(tubes, 'int6_outstage4nums.mp4', True)

FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [30]:
for t in tubes:
    print("Tube", tubes.index(t), "has length", len(t))
    
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])

Tube 0 has length 227
Tube 1 has length 23
Tube 2 has length 156
Tube 3 has length 114
Tube 4 has length 37
Tube 5 has length 35
Tube 6 has length 135
Tube 7 has length 75
Tube 8 has length 35
Tube 9 has length 36
Tube 10 has length 45
Tube 11 has length 44
Tube 12 has length 61
Tube 13 has length 89
Tube 14 has length 27
Tube 15 has length 88
Tube 16 has length 63
Tube 17 has length 80
Tube 18 has length 38


In [31]:
#Time to do some tube merging...

def getMerges(L, lookahead):
    mergeList = []
    for i in range(0,len(L)):
        CVx = [L[i][ind][2][0] for ind in range(-5,0)]
        CVy = [L[i][ind][2][1] for ind in range(-5,0)]
        forVx = sum(CVx)/len(CVx)
        forVy = sum(CVy)/len(CVy)  #ending velocity of tube i
        BVx = [L[i][ind][2][0] for ind in range(0,5)]
        BVy = [L[i][ind][2][1] for ind in range(0,5)]
        backVx = sum(BVx)/len(BVx)
        backVy = sum(BVy)/len(BVy)  #starting velocity of tube i

        forX = [L[i][ind][1][0][0] for ind in range(-10,0)]
        forY = [L[i][ind][1][0][1] for ind in range(-10,0)]
        backX = [L[i][ind][1][0][0] for ind in range(0,10)]
        backY = [L[i][ind][1][0][1] for ind in range(0,10)]
        
        forVx = (forX[-1] - forX[0]) /10
        forVy = (forY[-1] - forY[0]) /10
        backVx = (backX[-1] - backX[0]) /10
        backVy = (backY[-1] - backY[0]) /10

        predx = (sum(forX)/len(forX)) + (forVx*lookahead)
        predy = (sum(forY)/len(forY)) + (forVy*lookahead)
        predf = L[i][-1][0] + lookahead
        backpredx = (sum(backX)/len(backX)) - (backVx*lookahead)
        backpredy = (sum(backY)/len(backY)) - (backVy*lookahead)
        backpredf = L[i][0][0] - lookahead

        for j in range(0,len(tubes)):
            if i==j:
                continue

            closeones = [e for e in L[j] if abs(e[0] - predf)<5]
            Bcloseones= [e for e in L[j] if abs(e[0]-backpredf)<5]
            matchFlag = False
            BmatchFlag= False

            for e in closeones:
                r = math.sqrt((e[1][0][0] - predx)**2 + (e[1][0][1] - predy)**2 )
                if (r<100) and (abs(e[2][0] - forVx)<2) and (abs(e[2][1] - forVy)<2):
                    matchFlag = True

            for e in Bcloseones:
                r = math.sqrt((e[1][0][0] - backpredx)**2 + (e[1][0][1] - backpredy)**2 )
                if (r<50) and (abs(e[2][0] - backVx)<2) and (abs(e[2][1] - backVy)<2):
                    BmatchFlag = True

            if (matchFlag):
                mergeList.append([i,j,0])  #j goes in back of i
            if (BmatchFlag):
                mergeList.append([i,j,1])  #j goes in front of i

    nahList = []
    for m in mergeList:
        a = m[0]
        b = m[1]
        
        allVxa = [L[a][ind][2][0] for ind in range(0,len(L[a])) ] 
        Vxa = sum(allVxa)/len(allVxa)
        allVya = [L[a][ind][2][1] for ind in range(0,len(L[a])) ] 
        Vya = sum(allVya)/len(allVya)  

        allVxb = [L[b][ind][2][0] for ind in range(0,len(L[b])) ] 
        Vxb = sum(allVxb)/len(allVxb)
        allVyb = [L[b][ind][2][1] for ind in range(0,len(L[b])) ] 
        Vyb = sum(allVyb)/len(allVyb)

        if (Vxa/Vxb>0) and (Vya/Vyb>0 or (abs(Vya)<1.2 and abs(Vyb)<1.2) ) :
            A = [L[a][ind][0] for ind in range(0,len(L[a]))]
            B = [L[b][ind][0] for ind in range(0,len(L[b]))]
            overlap = [val for val in A if val in B]
            if len(overlap) > min(len(A),len(B))/2:
                nahList.append(m)
        else:
            nahList.append(m)

    mergeList = [m for m in mergeList if m not in nahList]
    
    return mergeList

def doMerge(L, mergeList):
    
    colorList = [t[0][3] for t in L]
    for m in mergeList:
        tlist = []
        for i in range(0,len(L)):
            for e in L[i]:
                if e[3]==colorList[m[0]] or e[3]==colorList[m[1]]:
                    tlist.append(i)

        flattubes = [ell for tube in L for ell in tube]
        cList = [colorList[a] for a in set(tlist)]
        connected = [e for e in flattubes if e[3] in cList]
        for i in set(tlist):
            L[i] = connected
    
    #Remove the extras (duplicates)
    delList = []
    for i in range(0,len(L)):
        if not(L.index(L[i]) == i):
            delList.append(i)
            
    print(delList)
    Otubes = [L[i] for i in range(0,len(L)) if i not in delList ]
    
    for t in Otubes:
        for e in t:
            e[3] = t[0][3]
    

    return Otubes


In [32]:
L = deepcopy(tubes)
lookahead = 5

M5 = getMerges(L,lookahead)
tubes = doMerge(L,M5)

#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)

[9, 10, 13]


In [33]:
putTubesOnVid(tubes, 'int6_outstage5.mp4')
putTubesOnVid(tubes, 'int6_outstage5nums.mp4', True)

FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [34]:
L = deepcopy(tubes)
lookahead = 60

M60 = getMerges(L,lookahead)
newtubes = doMerge(L,M60)
print(M60)

#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(newtubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)
    
putTubesOnVid(tubes, 'int6_outstage6.mp4')
putTubesOnVid(tubes, 'int6_outstage6nums.mp4', True)

[7, 10, 11, 12]
[[2, 7, 0], [7, 2, 1], [10, 6, 1], [11, 9, 1], [12, 7, 1]]
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [35]:
for t in tubes:
    t.sort(key = lambda x: x[0])

L = deepcopy(tubes)    
    
for t in tubes:
    if (len(t) >= (t[-1][0] - t[0][0] + 1)):
        continue
        
    else:
        print("Gaps in tube", tubes.index(t))
        toAdd = []
        have = [e[0] for e in t]
        donthave = [num for num in range(t[0][0], t[-1][0]) if num not in have]
        dlist = []
        for k, g in groupby(enumerate(donthave), lambda ix :ix[0]-ix[1]):
            dlist.append(list(map(itemgetter(1), g) ))
        for gap in dlist:
            preve = [e for e in t if e[0]==gap[0]-1][0]
            nexte = [e for e in t if e[0]==gap[-1]+1][0]
            fgap = len(gap)+1
            for f in gap:
                framediff = f - (gap[0]-1)
                
                X = (nexte[1][0][0] - preve[1][0][0]) * (framediff/fgap) + preve[1][0][0]
                Y = (nexte[1][0][1] - preve[1][0][1]) * (framediff/fgap) + preve[1][0][1]
                smallax = max(nexte[1][1][0],preve[1][1][0]) 
                bigax = max(nexte[1][1][1],preve[1][1][1]) 
                phi = (nexte[1][2] - preve[1][2]) * (framediff/fgap) + preve[1][2]
                Vx = (X - preve[1][0][0])/framediff
                Vy = (Y - preve[1][0][1])/framediff
                RGB = preve[3]
                
                newe = [f, ((X,Y), (smallax,bigax), phi), [Vx,Vy], RGB]
                toAdd.append(newe)
          
        t.extend(toAdd)  
            
for t in tubes:
    t.sort(key = lambda x: x[0])

    
putTubesOnVid(tubes, 'int6_outstage7.mp4')
putTubesOnVid(tubes, 'int6_outstage7nums.mp4', True)

Gaps in tube 2
Gaps in tube 6
Gaps in tube 8
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540
FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [36]:
#Putting into the familiar form. e = [x,y,a,b,phi,f,Vx,Vy,R,G,B, tubenum] 
#Prepping the data for event detection

tubeList=[]
L = deepcopy(tubes)
for tube in L:
    for e in tube:
        ellipse = [e[1][0][0], e[1][0][1], e[1][1][1]/2, e[1][1][0]/2, e[1][2], e[0], e[2][0], e[2][1], e[3][0], e[3][1], e[3][2],L.index(tube) ]
        tubeList.append(ellipse)

#print(tubeList)
DATA = np.array(tubeList)
print(DATA.shape)

(1562, 12)


In [37]:
len(tubes)

12

In [38]:
#This is code that I wrote a while back to detect "events"... when tubes get a little too close for comfort

sThresh = 100
tThresh = 3

eventList = []
#Each event is a "collision" of two tubes t1 and t2 given by [f1,x1,y1,t1,f2,x2,y2,t2]

for f in range(tThresh,F):
    inTime = [ (row[5],row[0],row[1],row[-1]) for row in DATA if abs(row[5] - f) < tThresh ]
    
    for comprow in inTime:
        notSameTube = np.array([row for row in inTime if row[-1] != comprow[-1]])
        if notSameTube.size!=0:
            checklocs = [ [ A[1], A[2]] for A in notSameTube]
            thisloc = np.array([comprow[1], comprow[2]])
            diff = np.linalg.norm((thisloc - checklocs),axis=1)

            closeIndex = [i for i,x in enumerate(diff) if x < sThresh]
            closeOnes = [notSameTube[i] for i in closeIndex]
            #print(closeOnes)

            for cL in closeOnes:
                CR = list(comprow)
                CR.extend(list(cL))
                eventList.append(CR)

                
majorEventList = []
# Each major event combines the large groups of events into a smaller format [fi, ff, xi, xf, yi, yf, t1, t2]
# where i,f are initial and final and location is an avg of the two tubes


datubes = np.sort([eventList[0][3], eventList[0][7]])
startf = int(eventList[0][0])
startloc = [np.mean((eventList[0][1],eventList[0][5])) , np.mean((eventList[0][2],eventList[0][6]))]

for i in range(len(eventList)):
    E = eventList[i]
    ts=np.sort([E[3], E[7]])
    

    if np.array_equal(ts,datubes):
        continue
    else:
        D=eventList[i-1]
        endf=int(D[0])
        endloc= [np.mean((D[1],D[5])) , np.mean((D[2],D[6]))]
        me = [startf, endf, startloc[0], endloc[0], startloc[1], endloc[1], int(datubes[0]), int(datubes[1])]
        #print(me)
        majorEventList.append(deepcopy(me))
        
        E = eventList[i]
        datubes=np.sort([E[3], E[7]])
        startf = int(E[0])
        startloc = [np.mean((E[1],E[5])) , np.mean((E[2],E[6]))]

Tlist = [ [me[-2],me[-1]] for me in majorEventList]
EventList = []

Tlistset = set(tuple(row) for row in Tlist)
for Tset in Tlistset:
    fList = []
    Tindices = [i for i,x in enumerate(Tlist) if tuple(x) ==Tset]
    for Ti in Tindices:    
        fList.append(majorEventList[Ti][0])
        fList.append(majorEventList[Ti][1])
    fmin = min(fList)
    fmax = max(fList)
    EventList.append([fmin, fmax, majorEventList[Tindices[0]][2], majorEventList[Tindices[-1]][2], majorEventList[Tindices[0]][4], majorEventList[Tindices[-1]][4], Tset[0], Tset[1] ])
    
        
eventlist = EventList
eventList = EventList
for e in EventList:
    print(e)
        

[17, 36, 671.3357238769531, 671.3357238769531, 320.7906951904297, 320.7906951904297, 1, 2]
[276, 299, 806.257985941569, 806.257985941569, 427.0464764912923, 427.0464764912923, 6, 7]
[465, 475, 794.9592224121094, 738.7648071289063, 388.9445078531901, 387.93519287109376, 10, 11]
[453, 475, 895.2574055989583, 756.1116790771484, 356.2738138834635, 346.2674903869629, 9, 10]
[465, 472, 797.9543863932292, 721.1518615722656, 391.4301452636719, 382.20625, 9, 11]
[106, 148, 623.2348876953125, 668.6202575683594, 320.4341654459636, 311.681298828125, 0, 5]
[113, 148, 682.5700454711914, 708.0654663085937, 331.3641418457031, 325.63689575195315, 2, 5]
[85, 224, 647.7077026367188, 711.5878434614701, 304.9584846496582, 306.8182927911932, 0, 2]
[185, 192, 652.0660339355469, 654.3245178222656, 329.5439971923828, 329.49866943359376, 0, 3]


In [39]:
putEventsOnVid(EventList,'int6_outstage7nums.mp4','int6_stage7events.mp4')

FRAME 100 / 539
FRAME 200 / 539
FRAME 300 / 539
FRAME 400 / 539
FRAME 500 / 539


In [40]:
#Creating a dictionary of information for each tube on the event list
eT = [e[-2] for e in EventList]
eT2= [e[-1] for e in EventList]
evTubes = set(eT + eT2)

evData = []
for tnum in evTubes:
    Ts = np.array([t[5] for t in tubeList if t[-1] == tnum])
    Xs = np.array([t[0] for t in tubeList if t[-1] == tnum])
    Ys = np.array([t[1] for t in tubeList if t[-1] == tnum])
    thisdict = {"tubenum": tnum, "Ts": Ts, "Xs": Xs, "Ys":Ys}
    evData.append(thisdict)

In [60]:
newtubes = deepcopy(tubes)
for e in EventList:
    dict1 = [d for d in evData if d['tubenum']==e[-2]][0]
    dict2 = [d for d in evData if d['tubenum']==e[-1]][0]
    
    #Gather data from before the event...
    T1 = dict1['Ts'][np.where(dict1['Ts']<e[0]-1)]
    T2 = dict2['Ts'][np.where(dict2['Ts']<e[0]-1)]
    X1 = dict1['Xs'][np.where(dict1['Ts']<e[0]-1)]
    X2 = dict2['Xs'][np.where(dict2['Ts']<e[0]-1)]
    Y1 = dict1['Ys'][np.where(dict1['Ts']<e[0]-1)]
    Y2 = dict2['Ys'][np.where(dict2['Ts']<e[0]-1)]

    if len(T1)<10 or len(T2)<10: #not enough info to do best fit
        continue
    
    print(e[-2])
    print(e[-1])
    #Get best fit equations
    XofT1 = np.polyfit(T1,X1,1)
    YofT1 = np.polyfit(T1,Y1,1)
    XofT2 = np.polyfit(T2,X2,1)
    YofT2 = np.polyfit(T2,Y2,1)
    print("First x(t) =", XofT1[0],"t + ", XofT1[1])
    print("First y(t) =", YofT1[0],"t + ", YofT1[1])
    print("Second x(t) =",XofT2[0],"t + ", XofT2[1])
    print("Second y(t) =",YofT1[0],"t + ", YofT2[1])
    toAdd1 = []
    toAdd2 = []
    prev1 = [ell for ell in newtubes[e[-2]] if ell[0]==e[0]][0]
    prev2 = [ell for ell in newtubes[e[-1]] if ell[0]==e[0]][0]
    #next1 = [ell for ell in newtubes[e[-2]] if ell[0]==e[1]][0]
    #next2 = [ell for ell in newtubes[e[-1]] if ell[0]==e[1]-5][0]
    
    #Make ghost tubes based on the best fit equations
    for f in range(e[0],e[0]+60):
        X1 = XofT1[0] * f + XofT1[1]
        Y1 = YofT1[0] * f + YofT1[1]
        smax1 = prev1[1][1][0]
        bgax1 = prev1[1][1][1]
        phi1 = prev1[1][2]
        Vx1 = XofT1[0]
        Vy1 = YofT1[0]
        RGB = [0,0,255]
        newe1 = [f, ((X1,Y1), (smax1,bgax1), phi1), [Vx1,Vy1], RGB]
        toAdd1.append(newe1)
        if f == e[0]:
            print(newe1)
            
        
        X2 = XofT2[0] * f + XofT2[1]
        Y2 = YofT2[0] * f + YofT2[1]
        smax2 = prev2[1][1][0]
        bgax2 = prev2[1][1][1]
        phi2 = prev2[1][2]
        Vx2 = XofT2[0]
        Vy2 = YofT2[0]
        RGB = [0,0,255]
        newe2 = [f, ((X2,Y2), (smax2,bgax2), phi2), [Vx2,Vy2], RGB]
        toAdd2.append(newe2)
        
    print()
    newtubes[e[-2]].extend(toAdd1)
    newtubes[e[-1]].extend(toAdd2)
    
        


6
7
First x(t) = 4.171749142280615 t +  -406.0775752299887
First y(t) = 0.17963792124552425 t +  372.8126969149828
Second x(t) = -3.4039928510972657 t +  1832.125320915808
Second y(t) = 0.17963792124552425 t +  326.5533771123637
[276, ((745.325188039461, 422.3927631787475), (133.22897338867188, 182.25875854492188), 163.2249989827474), [4.171749142280615, 0.17963792124552425], [0, 0, 255]]

0
2
First x(t) = 3.907052095776881 t +  286.62534333963407
First y(t) = -0.2316420996057589 t +  310.63003518128767
Second x(t) = -0.6991724958517479 t +  729.7837655122547
Second y(t) = -0.2316420996057589 t +  312.82683606721565
[85, ((618.724771480669, 290.9404567147982), (154.3766632080078, 259.7077941894531), 91.6922836303711), [3.907052095776881, -0.2316420996057589], [0, 0, 255]]

0
3
First x(t) = 2.082053592791485 t +  372.2220206124881
First y(t) = -0.1697224029329846 t +  307.07458299221713
Second x(t) = 4.095628780329287 t +  -167.23687610355788
Second y(t) = -0.1697224029329846 t +  315.1

In [65]:
putTubesOnVid(newtubes, 'int6_outstage8quad_last30.mp4')

FRAME 100 / 540
FRAME 200 / 540
FRAME 300 / 540
FRAME 400 / 540
FRAME 500 / 540


In [64]:
newtubes = deepcopy(tubes)
for e in EventList:
    dict1 = [d for d in evData if d['tubenum']==e[-2]][0]
    dict2 = [d for d in evData if d['tubenum']==e[-1]][0]
    
    #Gather data from before the event...
    T1 = dict1['Ts'][np.where(dict1['Ts']<e[0])]
    T2 = dict2['Ts'][np.where(dict2['Ts']<e[0])]
    X1 = dict1['Xs'][np.where(dict1['Ts']<e[0])]
    X2 = dict2['Xs'][np.where(dict2['Ts']<e[0])]
    Y1 = dict1['Ys'][np.where(dict1['Ts']<e[0])]
    Y2 = dict2['Ys'][np.where(dict2['Ts']<e[0])]

    if len(T1)<10 or len(T2)<10: #not enough info to do best fit
        continue
    
    print(e[-2])
    print(e[-1])
    #Get best fit equations
    XofT1 = np.polyfit(T1[-30:],X1[-30:],2)
    YofT1 = np.polyfit(T1[-30:],Y1[-30:],2)
    XofT2 = np.polyfit(T2[-30:],X2[-30:],2)
    YofT2 = np.polyfit(T2[-30:],Y2[-30:],2)
    print("First x(t) =", XofT1[0],"t^2 + ", XofT1[1],"t", + XofT1[2])
    print("First y(t) =", YofT1[0],"t^2 + ", YofT1[1],"t", + YofT1[2])
    print("Second x(t) =",XofT2[0],"t^2 + ", XofT2[1],"t", + XofT2[2])
    print("Second y(t) =",YofT2[0],"t^2 + ", YofT2[1],"t", + YofT2[2])
    print()
    toAdd1 = []
    toAdd2 = []
    prev1 = [ell for ell in newtubes[e[-2]] if ell[0]==e[0]-10][0]
    prev2 = [ell for ell in newtubes[e[-1]] if ell[0]==e[0]-10][0]

    #Make ghost tubes based on the best fit equations
    for f in range(e[0],e[0]+60):
        X1 = (XofT1[0] * f**2) + (XofT1[1]*f) + XofT1[2]
        Y1 = (YofT1[0] * f**2) + (YofT1[1]*f) + YofT1[2]
        smax1 = prev1[1][1][0]
        bgax1 = prev1[1][1][1]
        phi1 = prev1[1][2]
        Vx1 = XofT1[0]
        Vy1 = YofT1[0]
        RGB = [0,0,255]
        newe1 = [f, ((X1,Y1), (smax1,bgax1), phi1), [Vx1,Vy1], RGB]
        toAdd1.append(newe1)
        if f == e[0]:
            print(newe1)
        
        X2 = (XofT2[0] * f**2) + (XofT2[1]*f) + XofT2[2]
        Y2 = (YofT2[0] * f**2) + (YofT2[1]*f) + YofT2[2]
        smax2 = prev2[1][1][0]
        bgax2 = prev2[1][1][1]
        phi2 = prev2[1][2]
        Vx2 = XofT2[0]
        Vy2 = YofT2[0]
        RGB = [0,0,255]
        newe2 = [f, ((X2,Y2), (smax2,bgax2), phi2), [Vx2,Vy2], RGB]
        toAdd2.append(newe2)
    
    newtubes[e[-2]].extend(toAdd1)
    newtubes[e[-1]].extend(toAdd2)
    
        


6
7
First x(t) = 0.008452635510029194 t^2 +  0.2019662205348817 t 60.870714530710316
First y(t) = 0.03160578874065939 t^2 +  -15.661930741202571 t 2354.4482736467585
Second x(t) = -0.20803509655698185 t^2 +  104.23955568663433 t -12072.648051328793
Second y(t) = 0.01916492527474876 t^2 +  -9.973718312415647 t 1709.8356007063633

[276, ((760.5013540103215, 439.35795218331896), (90.12126159667969, 190.99203491210938), 164.14967346191406), [0.008452635510029194, 0.03160578874065939], [0, 0, 255]]
0
2
First x(t) = -0.004994106883750236 t^2 +  3.1911354265889265 t 354.3711789374131
First y(t) = 0.006739995378427406 t^2 +  -1.2379439923844153 t 347.21914270377727
Second x(t) = 0.023512138990257975 t^2 +  -2.3024130453509266 t 726.8662072065958
Second y(t) = -0.020562850540551958 t^2 +  2.810607391282526 t 219.64270096629747

[85, ((589.5352679623763, 290.69036996024), (166.16241455078125, 184.55332946777344), 175.56417846679688), [-0.004994106883750236, 0.006739995378427406], [0, 0, 255]]
0


In [50]:
print(T1[-30:])

[155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172
 173 174 175 176 177 178 179 180 181 182 183 184]
